## 프롬프트 엔지니어링
___

* 라이브러리 로드

In [1]:
!python -m pip install yfinance

  Using cached yfinance-0.2.40-py2.py3-none-any.whl.metadata (11 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached lxml-5.2.2-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.4 kB)
  Using cached frozendict-2.4.4-py312-none-any.whl.metadata (23 kB)
  Using cached peewee-3.17.5-py3-none-any.whl
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
Using cached yfinance-0.2.40-py2.py3-none-any.whl (73 kB)
Using cached frozendict-2.4.4-py312-none-any.whl (16 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached lxml-5.2.2-cp312-cp312-macosx_10_9_universal2.whl (8.2 MB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)


In [1]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from utils.exchange import convert_usd_to_krw
from langchain.chains import LLMChain, SequentialChain
from langchain.output_parsers import PydanticOutputParser
from pprint import pprint
from dotenv import load_dotenv
load_dotenv()

Python-dotenv could not parse statement starting at line 30
Python-dotenv could not parse statement starting at line 30


True

* OpenAI API서비스는 상용 언어모델 서비스로 토큰당 청구되는 유료 서비스이다.
* 가격표 (출처 : https://openai.com/api/pricing/)
  
  * 대표모델 API 사용료 ( 24/5/19 현재 )
   
     | 모델명                  | 1M 토큰당 가격 (입력) | 1M 토큰당 가격 (출력) |
     |-------------------------|-----------------------|-----------------------|
     | gpt-3.5-turbo-0125      | $0.5                  | $1.5                  |
     | gpt-3.5-turbo-instruct  | $1.5                  | $2                    |
     | gpt-4o                  | $5                    | $15                   |
     | gpt-4 turbo             | $10                   | $30                   |
     | gpt-4                   | $30                   | $60                   |
     | gpt-4 32k               | $60                   | $120                  |

  <br/>
  
  * Fine-Tuning모델 API 사용료 (24/5/19 현재)
   
     | 모델명         | 훈련 비용 (입력) | 사용 비용 (입력) | 사용 비용 (출력) |
     |----------------|------------------|------------------|------------------|
     | gpt-3.5-turbo  | $8.00 / 1M 토큰  | $3.00 / 1M 토큰  | $6.00 / 1M 토큰  |
     | davinci-002    | $6.00 / 1M 토큰  | $12.00 / 1M 토큰 | $12.00 / 1M 토큰 |
     | babbage-002    | $0.40 / 1M 토큰  | $1.60 / 1M 토큰  | $1.60 / 1M 토큰  |


  <br/>
  
   * 임베딩모델 API 사용료 (24/5/19 현재)
      
        | 모델명                  | 1M 토큰당 가격       |
        |-------------------------|----------------------|
        | text-embedding-3-small  | $0.02                |
        | text-embedding-3-large  | $0.13                |
        | ada v2                  | $0.10                |





In [2]:
# 언어모델 로드 
gpt35 = ChatOpenAI(model="gpt-3.5-turbo-0125")

/Users/dante/miniconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


___
* 제로샷
  
  * 방식: 어떤 예제도 보여주지 않고 요청사항을 직접적으로 전달하는 방식
  
  * 핵심 개념: 기본 모델의 훈련 체크포인트에 전적으로 의존한다.
  * 특징: 모델이 해당 주제에 대한 충분한 정보를 갖고 있지 않을 수 있으므로, 답변의 정확성이 떨어질 수 있음

In [3]:
prompt = PromptTemplate(
    template="{category} 에 해당하는 직무의 종류는 무엇이 있나요? {num}개를 소개해주세요.",
    input_variables=["category", "num"]
)
chain = prompt | gpt35

with get_openai_callback() as cb:
    resp = chain.invoke({"category": "데이터", "num": 10})
    pprint(resp.content)
    convert_usd_to_krw(cb.total_cost)


('1. 데이터 분석가 (Data Analyst)\n'
 '2. 데이터 과학자 (Data Scientist)\n'
 '3. 비즈니스 인텔리전스 애널리스트 (Business Intelligence Analyst)\n'
 '4. 데이터 엔지니어 (Data Engineer)\n'
 '5. 데이터 웨어하우스 전문가 (Data Warehouse Specialist)\n'
 '6. 데이터 마이닝 전문가 (Data Mining Specialist)\n'
 '7. 데이터 시각화 전문가 (Data Visualization Specialist)\n'
 '8. 빅데이터 아키텍트 (Big Data Architect)\n'
 '9. 데이터베이스 관리자 (Database Administrator)\n'
 '10. 인공지능 엔지니어 (AI Engineer)')
┌──────────────────────────────┐
│          0.50원 사용          │
└──────────────────────────────┘


___
* 퓨샷
  
    * 방식: 명시적인 지침이 없이 요청사항과 관련된 몇가지 입력과 출력 예제를 제공
    
    * 핵심개념 : 요청내용의 의도와 목표를 예제로부터 자연스럽게 추론하도록 유도
    * 특징 : 퓨샷은 추론 능력을 향상시키지만, 데이터 변동성에 취약함. 명시적 지침을 추가하는 방법으로 더 개선이 가능함.

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    template="{question} -> {answer}",
    input_variables=["question", "answer"]
)

examples = [
    {"question": "데이터 분석가는 어떤 일을 하나요?", "answer": "데이터 분석가는 데이터를 수집, 처리, 분석하여 유의미한 인사이트 도출"},
    {"question": "데이터 엔지니어는 어떤 일을 하나요?", "answer": "데이터 엔지니어는 데이터 파이프라인을 설계하고 구축하여 데이터를 효율적으로 처리하고 저장함."}
]

prompt = FewShotPromptTemplate(
    prefix="다음은 데이터 직무에 대한 예시입니다:",
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {question}",
    input_variables=["question"]
)

chain = prompt | gpt35
with get_openai_callback() as cb:
    resp = chain.invoke({"question": "AI 모델러는 어떤 일을 하나요?"})
    pprint(resp.content)
    convert_usd_to_krw(cb.total_cost)


('AI 모델러는 인공지능 모델을 개발하고 최적화하여 다양한 데이터 소스로부터 패턴을 발견하고 예측 모델을 구축합니다. 또한 모델의 정확성과 '
 '성능을 향상시키기 위해 테스트 및 평가를 수행하며, 실제 시스템에 통합하여 실시간 예측을 가능하게 합니다.')
┌──────────────────────────────┐
│          0.37원 사용          │
└──────────────────────────────┘


___
* Self-Consistency Prompting

    * 방식 : 질문에 대해 여러 후보 답변을 생성하고 가장 일관되거나 가장 빈번한 답변이 최종 답변으로 채택
  
    * 특징: 팩트체크 또는 정보 통합이 필요한 경우 활용됨

In [93]:
solutions_prompt = PromptTemplate(
    template="""다음 요청 사항에 대한 {count}개의 중복되지 않는 답변을 생성하세요 : 
    {request}
    * 응답 목록 :
    """,
    input_variables=["count", "request"]
)

solutions_chain = LLMChain(
    llm=gpt35,
    prompt=solutions_prompt,
    output_key="solutions"
)


/Users/dante/miniconda3/envs/langchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [96]:
consistency_prompt = PromptTemplate(
    template="""
    * 제시된 응답 목록: {solutions}
    * 요청 : 각 응답에 대해 몇번이나 발생할지 카운팅을 해봐. 그리고, 그중에서 가장 많이 등장하는 답변을 선택해줘
    * 가장 많이 등장하는 응답:
    """,
    input_variables=["solutions"]
)

consistency_chain = LLMChain(
    llm=ChatOpenAI(model="gpt-3.5-turbo-0125"),
    prompt=consistency_prompt,
    output_key="final_solution"
)

In [98]:
answer_chain = SequentialChain(
    chains=[solutions_chain, consistency_chain],
    input_variables=["count", "request"],
    output_variables=["final_solution"]
)

In [104]:
resp = answer_chain.run(count=5, request="데이터 분석가는 분석만 잘하면 되는거야? 엔지니어링은 몰라도되지?")
pprint(resp)

('3. 데이터 분석가는 분석 능력 뿐만 아니라 엔지니어링 지식도 필요합니다. 데이터를 수집하고 처리하는 과정을 이해하면 분석 작업이 '
 '원활해집니다. (2번)')


___
* CoT (Chain-of-Thought) Prompting

  * 방식: 문제 해결 과정을 단계별로 설명하면서 답을 도출하는 기법
  
  * 핵심 개념: 모델이 논리적 사고 과정을 거치도록 하여 복잡한 문제를 해결하도록 유도
  * 특징: 제로샷 CoT, 퓨샷 CoT 으로 응용이 가능함.


___
* MinToMax Prompting

    * 방식 및 개념: 단순한 질문부터 시작하여 점차 복잡한 질문으로 진행하는 기법
  
    * 특징 : 단계별 난이도 설정이 중요하며, 초기 단계에서 모델이 충분히 학습할 수 있도록 간단한 질문을 제공해야함.

In [8]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

1. 중학교 수학 문제 예시
  
    <img src="./images/formula1.png" width="300">

* 문제 식 : $\lim_{x \to 0} \frac{\tan\left(\frac{x}{5}\right)}{\sin(2x)}$


In [26]:
formula = r"\[\lim_{x \to 0} \frac{\tan\left(\frac{x}{5}\right)}{\sin(2x)}\]"

In [27]:
prompt = PromptTemplate(
    template="다음 문제의 해답을 찾아주세요 : {formula}",
    input_variables=["formula"]
)

resp = (prompt | gpt35).invoke({"formula": formula})
pprint(resp.content)


('먼저 주어진 함수를 0/0 형태로 변형하기 위해 미분의 정의를 사용합니다.\n'
 '\n'
 '\\[\\lim_{x \\to 0} \\frac{\\tan\\left(\\frac{x}{5}\\right)}{\\sin(2x)} = '
 '\\lim_{x \\to 0} '
 '\\frac{\\frac{\\tan\\left(\\frac{x}{5}\\right)}{\\frac{x}{5}}}{\\frac{\\sin(2x)}{2x}}\\]\n'
 '\n'
 '여기서 분자와 분모의 극한값을 구하기 위해 미분의 정의인 \\(\\lim_{t \\to 0} \\frac{\\tan(t)}{t} = '
 '1\\)과 \\(\\lim_{t \\to 0} \\frac{\\sin(t)}{t} = 1\\)을 사용합니다.\n'
 '\n'
 '따라서 위의 식은 다음과 같이 변형됩니다.\n'
 '\n'
 '\\[ = \\frac{1}{2} \\cdot 1 = \\frac{1}{2}\\]\n'
 '\n'
 '따라서 \\(\\lim_{x \\to 0} \\frac{\\tan\\left(\\frac{x}{5}\\right)}{\\sin(2x)} '
 '= \\frac{1}{2}\\)입니다.')


In [64]:
memory = ConversationBufferMemory()
streaming_llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, streaming=True)

In [65]:
chain = ConversationChain(llm=streaming_llm, memory=memory)
chain.predict(input=r"\lim_{x \to 0} \frac{\tan\left(\frac{x}{5}\right)}{\sin(2x)} 문제를 풀어보려고해. 분자의 식은 뭐지? 그리고 x->0일때, 그 식의 작은 근사식은 뭐지?")

'분자의 식은 tan(x/5)이고, x->0일때, tan(x)는 x에 근사하므로 tan(x/5)는 x/5에 근사합니다. 따라서 분자의 근사식은 x/5입니다.'

In [66]:
chain.predict(input="그러면 분모식은 뭐지? x->0 일때, 그 식의 작은 근사식은 뭐지?")

'분모의 식은 sin(2x)이고, x->0일때, sin(x)는 x에 근사하므로 sin(2x)는 2x에 근사합니다. 따라서 분모의 근사식은 2x입니다.'

In [67]:
chain.predict(input="분자의 작은 근사식을 분모의 작은 근사식으로 나누면 어떻게 되는 거지?")


'분자의 근사식인 x/5를 분모의 근사식인 2x로 나누면, (x/5) / (2x) = 1/10 가 됩니다.'

In [68]:
chain.predict(input="그러면 전체식의 극한은 어떻게 되는 거지?")


'전체식의 극한은 분자의 근사식을 분모의 근사식으로 나눈 값인 1/10이 됩니다. 따라서 \\lim_{x \\to 0} \\frac{\\tan\\left(\\frac{x}{5}\\right)}{\\sin(2x)} = 1/10 입니다.'

In [69]:
memory.chat_memory.messages

[HumanMessage(content='\\lim_{x \\to 0} \\frac{\\tan\\left(\\frac{x}{5}\\right)}{\\sin(2x)} 문제를 풀어보려고해. 분자의 식은 뭐지? 그리고 x->0일때, 그 식의 작은 근사식은 뭐지?'),
 AIMessage(content='분자의 식은 tan(x/5)이고, x->0일때, tan(x)는 x에 근사하므로 tan(x/5)는 x/5에 근사합니다. 따라서 분자의 근사식은 x/5입니다.'),
 HumanMessage(content='그러면 분모식은 뭐지? x->0 일때, 그 식의 작은 근사식은 뭐지?'),
 AIMessage(content='분모의 식은 sin(2x)이고, x->0일때, sin(x)는 x에 근사하므로 sin(2x)는 2x에 근사합니다. 따라서 분모의 근사식은 2x입니다.'),
 HumanMessage(content='분자의 작은 근사식을 분모의 작은 근사식으로 나누면 어떻게 되는 거지?'),
 AIMessage(content='분자의 근사식인 x/5를 분모의 근사식인 2x로 나누면, (x/5) / (2x) = 1/10 가 됩니다.'),
 HumanMessage(content='그러면 전체식의 극한은 어떻게 되는 거지?'),
 AIMessage(content='전체식의 극한은 분자의 근사식을 분모의 근사식으로 나눈 값인 1/10이 됩니다. 따라서 \\lim_{x \\to 0} \\frac{\\tan\\left(\\frac{x}{5}\\right)}{\\sin(2x)} = 1/10 입니다.')]

* 두번째 문제

In [70]:
prompt = PromptTemplate(
    template="다음 문제의 해답을 찾아주세요 : {quiz}",
    input_variables=["quiz"]
)

In [72]:
quiz = """
    도둑을 쫓는 주인이 처음에는 도둑에게 37 리 뒤처져 있었습니다. 주인이 145 리를 이동한 후, 
    그는 이제 도둑에게 23 리 뒤처져 있었습니다. 따라서 그는 도둑에게 37 - 23 = 14 리를 따라잡은 것입니다.
    주인이 145 리를 이동하여 14 리를 따라잡았다면, 
    23 리를 더 따라잡기 위해서는 몇 리를 더 이동해야 하나요?
"""

In [71]:
# 틀린 답을 말하는 GPT 3.5 turbo 모델
chain = prompt | gpt35
resp = chain.invoke({"quiz": quiz})
pprint(resp.content)

('주인이 145 리를 이동하여 14 리를 따라잡았으므로, 도둑과의 거리는 145 - 14 = 131 리 입니다.\n'
 '따라서 주인이 23 리를 더 따라잡기 위해서는 131 + 23 = 154 리를 더 이동해야 합니다. \n'
 '즉, 주인은 총 154 리를 더 이동해야 도둑을 23 리 앞서게 됩니다.')


In [73]:
# 최신 멀티모달 모델인 GPT 4 옴니의 경우 문제의 정답을 말해준다.
# 프롬프트 공학을 하지 않아도, 모델이 자체적으로 
gpt4o = ChatOpenAI(model='gpt-4o')
chain = prompt | gpt4o
resp = chain.invoke({"quiz": quiz})
pprint(resp.content)

('문제를 해결하기 위해 다음과 같은 단계를 통해 접근할 수 있습니다.\n'
 '\n'
 '1. 주인이 145 리를 이동하여 14 리를 따라잡았다는 사실로부터, 주인이 도둑을 따라잡는 속도를 계산할 수 있습니다.\n'
 '2. 주인이 14 리를 따라잡는 데 145 리를 이동했다면, 주인의 따라잡는 속도는 145 리/14 리 입니다.\n'
 '3. 이제 주인이 23 리를 더 따라잡기 위해 몇 리를 더 이동해야 하는지 계산합니다.\n'
 '\n'
 '따라잡는 속도를 계산하면:\n'
 '\\[ \\text{따라잡는 속도} = \\frac{145 \\text{ 리}}{14 \\text{ 리}} \\]\n'
 '\n'
 '주인이 23 리를 따라잡기 위해 이동해야 하는 거리는:\n'
 '\\[ \\text{이동 거리} = \\frac{145 \\text{ 리}}{14 \\text{ 리}} \\times 23 \\text{ '
 '리} \\]\n'
 '\n'
 '이 계산을 하면:\n'
 '\\[ \\text{이동 거리} =  \\frac{145}{14} \\times 23 \\]\n'
 '\\[ \\text{이동 거리} = 10.3571 \\times 23 \\]\n'
 '\\[ \\text{이동 거리} = 238.2143 \\text{ 리} \\]\n'
 '\n'
 '따라서, 주인이 23 리를 더 따라잡기 위해서는 약 238.2143 리를 더 이동해야 합니다. \n'
 '\n'
 '결론적으로, 주인은 도둑을 따라잡기 위해 238.2143 리를 더 이동해야 합니다.')


In [83]:
# GPT3.5 turbo 모델을 프롬프트 공학으로 재시도했으나 실패
chain = prompt | gpt35

resp = chain.invoke(input=f"""
    * 퀴즈를 한번에 풀지 말고, 한단계씩 요청할거야.
    * 퀴즈내용: {quiz}
    * 첫번째 요청: 주인의 속도를 구해봐
    * 두번째 요청: 23리를 움직이려면 지금 속도로 몇 리를 가야하니?
    * 두번째 요청을 최종 답변으로 말해줘
""")
pprint(resp.content)

('* 첫번째 요청: 주인의 속도는 (37 - 23) 리를 145 리의 이동거리로 이동하는데 걸린 시간으로 구할 수 있습니다. \n'
 '    속도 = 이동거리 / 시간 = 122 / 145 = 0.841 리/시간\n'
 '* 두번째 요청: 23 리를 따라잡기 위해서는 (37 - 23) 리를 23 리의 이동거리로 이동하는데 걸리는 시간을 구해야 합니다. \n'
 '    시간 = 이동거리 / 속도 = 14 / 0.841 = 16.64 시간\n'
 '* 최종 답변: 23 리를 더 따라잡기 위해서는 16.64 시간을 더 이동해야 합니다.')


In [75]:
# GPT 3.5 turbo로 문제를 메모리 버퍼를 사용해 작은 문제에서 복잡한 문제로 단계적으로 질의하며 문제해결
memory = ConversationBufferMemory()

In [76]:
chain = ConversationChain(llm=gpt35, memory=memory)

In [78]:
chain.predict(input=f"""
    * 퀴즈를 한번에 풀지 말고, 한단계씩 요청할거야.
    * 퀴즈내용: {quiz}
    * 첫번째 요청: 주인의 속도를 구해봐
""")

'첫번째 요청에 대해 답변드리겠습니다. 주인이 145리를 이동하여 14리를 따라잡았으므로, 주인의 속도는 14리를 145리로 나눈 것이 됩니다. 따라서 주인의 속도는 14 / 145 = 0.0967 리/리 입니다. 이 부분에 대해 추가 질문이 있으실 경우 언제든지 물어보세요!'

In [79]:
chain.predict(input=f"""
    * 두번째 요청: 23리를 움직이려면 지금 속도로 몇 리를 가야하니?
""")

'두번째 요청에 대해 답변드리겠습니다. 주인이 현재 속도인 0.0967 리/리로 23리를 더 따라잡기 위해서는 다음과 같이 계산할 수 있습니다. 주인이 23리를 따라잡으려면 23를 주인의 현재 속도로 나눈 값을 더해야 합니다. 따라서 23 / 0.0967 ≈ 237.73 리를 더 이동해야 23리를 따라잡을 수 있습니다. 이 부분에 대해 추가 질문이 있으실 경우 언제든지 물어보세요!'

___
* CoD(Chain-of-Density)

    * 방식 및 개념: 생성된 요약의 정보 밀도를 점진적으로 높이면서 길이를 제어하는 프롬프트 가이드
  
    * 특징: 요약에서 정보의 밀도를 향상시키는 효과

In [84]:
# 제로샷 시도
article = open("dataset/articles/article1.txt").read()
prompt = PromptTemplate(
    template="다음 아티클을 요약 및 정리해줘 : {article}",
    input_variables=["article"]
)
chain = prompt | gpt35
with get_openai_callback() as cb:
    resp = chain.invoke({"article": article})
    pprint(resp.content)
    convert_usd_to_krw(cb.total_cost)

("문화체육관광부가 발표한 '2023년 국민 독서실태' 결과에 따르면 성인의 독서율과 독서량이 계속해서 하락하고 있으며, 성인 10명 중 "
 '6명은 1년 동안 아무 책도 읽지 않은 상황이다. 이러한 문제는 문해력 저하로 이어질 수 있으며, 문해력은 소통과 연결고리를 이루는 '
 '중요한 역할을 한다. 특히 비즈니스 현장에서는 문해력이 업무 능력과 직결되어 있어 중요하다. \n'
 '\n'
 '문해력을 향상시키기 위해서는 많이 읽고, 많이 쓰고, 많이 생각하는 것이 중요하다. 회사는 직원을 대상으로 문해력 향상 프로그램을 '
 '제공하고, 독서 모임을 추천하며, 문서 작성을 위한 공통적인 매뉴얼을 마련하는 등의 전략을 펼쳐야 한다. 개인의 노력뿐만 아니라 회사와 '
 '국가 차원에서도 적극적인 대응이 필요하다. 함께하는 힘이 필요한 시대이며, 문해력은 개인의 문제가 아닌 전체 사회의 문제로 인식되어야 '
 '한다.')
┌──────────────────────────────┐
│          2.59원 사용          │
└──────────────────────────────┘


In [85]:
# CoD 프롬프팅 시도
# ref : https://arxiv.org/abs/2309.04269
prompt = PromptTemplate(
    template="""
    Article: {article}
    You will generate increasingly concise, entity-dense summaries of the above article. 

    Repeat the following 2 steps 5 times. 

    Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
    Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

    A missing entity is:
    - relevant to the main story, 
    - specific yet concise (5 words or fewer), 
    - novel (not in the previous summary), 
    - faithful (present in the article), 
    - anywhere (can be located anywhere in the article).

    Guidelines:

    - The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
    - Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
    - Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
    - The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
    - Missing entities can appear anywhere in the new summary.
    - Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 
    - In korean

    Remember, use the exact same number of words for each summary.
    """,
    input_variables=["article"]
)
chain = prompt | gpt35
with get_openai_callback() as cb:
    resp = chain.invoke({"article": article})
    pprint(resp.content)
    convert_usd_to_krw(cb.total_cost)


('Missing entities: 문해력; 독서실태; 독서지표\n'
 '\n'
 'Summary: 성인 문해력 저하 현상 심각. 문해력 논란 지속, 성인 독서율 하락. 성인 10명 중 6명 1년 동안 책 읽지 않음. '
 '문해력은 소통 중요, 글의 맥락 이해 필요.문해력은 소통을 위한 능력. OECD 조사 성인 문해력 중요성 강조. 성인 문해력 업무 능력과 '
 '직결. 문해력 높으면 업무 정확, 빠르게 처리.회사의 전략 필요, 직원 문해력 향상 프로그램 제공. 회사 내 독서 모임 추천. 회사에서 '
 '사용할 수 있는 공통적인 매뉴얼 필요. 함께 하는 힘이 필요한 시대.')
┌──────────────────────────────┐
│          2.50원 사용          │
└──────────────────────────────┘


In [86]:
# gpt4 옴니 제로샷 시도
zeroshot_prompt = PromptTemplate(
    template="다음 아티클을 요약 및 정리해줘 : {article}",
    input_variables=["article"]
)
chain = zeroshot_prompt | gpt4o
with get_openai_callback() as cb:
    resp = chain.invoke({"article": article})
    pprint(resp.content)
    convert_usd_to_krw(cb.total_cost)

('### 요약\n'
 '\n'
 '문화체육관광부가 발표한 ‘2023년 국민 독서실태’ 결과에 따르면 성인의 독서율과 독서량이 지속적으로 감소하고 있으며, 10명 중 6명은 '
 '1년 동안 책을 읽지 않았다. 문해력 저하로 인한 사회적 파장은 크며, 이는 소통의 문제를 야기한다. 문해력은 단순히 글을 읽고 쓸 수 '
 '있는 능력을 넘어, 글의 맥락을 이해하고 의도를 명확하게 전달하는 능력을 포함한다. 성인의 문해력은 업무 효율성과 직결되며, 세대 간 '
 '문해력 차이로 인해 직장에서의 소통 문제가 발생하기도 한다.\n'
 '\n'
 '비즈니스 현장에서 문해력은 중요한데, 보고서 작성과 같은 업무에서 문해력이 부족하면 소통의 어려움과 갈등이 초래된다. 이를 해결하기 '
 '위해서는 회사 차원의 전략적 접근이 필요하다. 전문가들은 ‘다독, 다작, 다상량’을 권장하지만, 시간을 내기 어려운 현실에서 회사의 '
 '지원이 필요하다.\n'
 '\n'
 '회사는 문해력 향상 프로그램을 제공하고, 독서 모임을 조직하며, 문서 작성 매뉴얼을 마련해야 한다. 이는 조직의 성과와 직결되며, '
 '장기적으로 회사의 발전에 도움이 된다. 문해력 문제는 개인의 책임으로 돌릴 수 없으며, 회사와 국가의 적극적인 대응이 필요하다.\n'
 '\n'
 '### 주요 내용 정리\n'
 '\n'
 '1. **독서 실태와 문해력 저하**:\n'
 '   - 성인의 독서율, 독서량, 독서 시간이 감소하고 있음.\n'
 '   - 성인 10명 중 6명이 1년 동안 책을 읽지 않음.\n'
 '   - 문해력 저하로 인한 사회적 소통 문제 발생.\n'
 '\n'
 '2. **문해력의 중요성**:\n'
 '   - 단순한 글 읽기, 쓰기를 넘어 맥락 이해와 의도 전달 능력 포함.\n'
 '   - 성인의 문해력은 업무 능력과 직결되며, 세대 간 문해력 차이로 직장 내 소통 문제 발생.\n'
 '\n'
 '3. **비즈니스 현장에서의 필요성**:\n'
 '   - 문해력 부족으로 인한 보고서 작성 

In [87]:
# gpt4 옴니 CoD 시도
chain = prompt | gpt4o
with get_openai_callback() as cb:
    resp = chain.invoke({"article": article})
    pprint(resp.content)
    convert_usd_to_krw(cb.total_cost)

('**Step 1:**\n'
 '1-3 informative entities: "문화체육관광부"; "성인 독서율"; "문해력 저하"\n'
 '\n'
 '**Step 2:**\n'
 '이 기사는 성인들의 문해력 저하 현상과 관련된 다양한 문제들을 다룹니다. 문화체육관광부가 발표한 2023년 국민 독서실태에 따르면 성인 '
 '독서율이 낮아지고 있으며, 이는 문해력 저하로 이어지고 있습니다. 문해력 저하는 사회적 소통에 큰 영향을 미치며, 비즈니스 현장에서도 '
 '중요한 문제로 부각되고 있습니다. 성인의 문해력을 향상시키기 위한 다양한 전략이 필요합니다.\n'
 '\n'
 '---\n'
 '\n'
 '**Step 1:**\n'
 '1-3 informative entities: "1년 동안 단 1권의 책도 읽지 않은"; "소통"; "보고서"\n'
 '\n'
 '**Step 2:**\n'
 '문화체육관광부가 발표한 2023년 국민 독서실태에 따르면 성인 독서율이 낮아지고 있으며, 성인 10명 중 6명은 1년 동안 단 1권의 '
 '책도 읽지 않았습니다. 이는 문해력 저하로 이어지고, 사회적 소통에 큰 영향을 미칩니다. 비즈니스 현장에서도 문해력은 중요하며, 보고서 '
 '작성과 같은 업무 능력과 직결됩니다. 성인의 문해력을 향상시키기 위한 다양한 전략이 필요합니다.\n'
 '\n'
 '---\n'
 '\n'
 '**Step 1:**\n'
 '1-3 informative entities: "OECD"; "MZ세대"; "독서 모임"\n'
 '\n'
 '**Step 2:**\n'
 '2023년 국민 독서실태에 따르면 성인 10명 중 6명은 1년 동안 단 1권의 책도 읽지 않았습니다. 이는 문해력 저하로 이어지며, '
 '사회적 소통과 비즈니스 현장에 영향을 미칩니다. OECD 조사에 따르면 한국은 세대 간 문해력 간격이 크며, 이는 MZ세대가 대기업을 '
 '떠나는 이유 중 하나입니다. 독서 모임과 같은 전략이 성인의 문해력 향상에 필요합니다.\n'
 '\n'
 '--

___
* CoV (Chain-of-Verification)

설명: 답변을 단계별로 검증하는 기법입니다.
핵심 개념: 각 단계에서의 논리적 일관성을 검증하며 최종 답을 도출합니다.
고려사항: 각 단계의 검증 과정을 철저히 수행해야 하며, 모든 단계가 정확해야 최종 답변의 신뢰성이 보장됩니다.

In [88]:
# gpt3.5 turbo 제로샷
zeroshot_prompt = PromptTemplate(
    template="다음에 대해 아는 것을 말해줘 : {topic}",
    input_variables=["topic"]
)
chain = zeroshot_prompt | gpt35
with get_openai_callback() as cb:
    resp = chain.invoke({"topic": "BTS"})
    pprint(resp.content)
    convert_usd_to_krw(cb.total_cost)

('BTS는 대한민국의 7인조 보이 그룹으로, 방탄소년단 (Bangtan Sonyeondan)의 줄임말입니다. 그들은 2013년에 데뷔하여 '
 '국내외에서 큰 인기를 얻고 있는 그룹으로, 음악적인 업적 뿐만 아니라 사회공헌활동과 세계적인 영향력으로도 유명합니다. 그들은 다양한 음악 '
 '장르를 넘나들며 새로운 시도와 창의적인 음악으로 팬들을 매료시키고 있습니다. 또한, 그들은 UNICEF와 함께한 Love Myself '
 '캠페인을 통해 청소년들에게 긍정적인 메시지를 전달하고 있습니다. BTS는 K-pop 산업을 세계적으로 선도하는 그룹으로 평가받고 '
 '있습니다.')
┌──────────────────────────────┐
│          0.56원 사용          │
└──────────────────────────────┘


In [92]:
# gpt3.5 turbo CoV (Chain-of-Verification)
cov_prompt = PromptTemplate(
    template="""
    1. 다음 주제에 대해 아는 것을 생성해봐 : {topic}. 
    2. 그리고 그 정보가 맞는지 검증하고 부족한 내용을 채우거나 잘못된 부분을 수정해줘.
    3. 내용을 요약하지 말고 처음 생성한 분량만큼 풍성하게 작성해줘. 
    3. 수정된 최종 내용만 답변해줘.
    """,
    input_variables=["topic"]
)
chain = cov_prompt | gpt35
with get_openai_callback() as cb:
    resp = chain.invoke({"topic": "BTS"})
    pprint(resp.content)
    convert_usd_to_krw(cb.total_cost)

('BTS는 한국의 보이 그룹으로, 방탄소년단(防彈少年團)의 줄임말이다. 2013년에 데뷔한 이후 꾸준한 인기를 얻어 세계적으로 유명한 '
 '아이돌 그룹으로 성장했다. BTS는 RM, 진, 슈가, 제이홉, 지민, 뷔, 정국으로 구성되어 있으며, 각 멤버들은 뛰어난 보컬 실력과 '
 '댄스 실력을 가지고 있다.\n'
 '\n'
 "BTS는 음악뿐만 아니라 사회 문제에 대해 다뤄 노래하는 등의 특징을 가지고 있어, 팬들 사이에서는 '힙합몬스터', '천재 아이돌' 등의 "
 '별명으로 불린다. 그들의 음악은 청소년들을 위로하고, 다양한 메시지를 전달하며 세계 각국의 팬들로부터 사랑을 받고 있다. 또한, '
 'UNICEF와 함께 한 LOVE MYSELF 캠페인을 통해 아동학대 방지와 자해 예방을 위한 활동도 펼치고 있다. \n'
 '\n'
 'BTS는 한국뿐만 아니라 전 세계적으로 많은 상을 수상하며 K-pop의 대표적인 아티스트로 자리매김했다. 그들의 음악은 언어의 장벽을 '
 '넘어 다양한 국가와 문화에서 사랑받고 있으며, 많은 이들에게 영감을 주고 희망을 전하고 있다. BTS는 그들만의 색깔과 메시지를 지닌 '
 '그룹으로서 음악적으로나 사회적으로 큰 영향력을 행사하고 있다.')
┌──────────────────────────────┐
│          1.18원 사용          │
└──────────────────────────────┘


___
* ToT (Tree-of-Thought) Prompting

    * 방식 및 개념: 요청사항을 위한 다양한 경로를 탐색하고 각 경로의 결과를 비교 평가 후 최적의 추론 경로를 찾음
  
    * 특징: 복잡한 문제에서 특히 유용



In [3]:
solution_prompt = PromptTemplate(
    template="""
    다음 제시된 문제를 {count}가지의 고유한 해결 방법을 제시해줘. 함께 제시된 요소들을 고려해서 진행해줘.
    * 문제: {problem}
    * 요소: {factors}
    """,
    input_variables=["count", "problem", "factors"]
)

solution_chain = LLMChain(llm=gpt35, prompt=solution_prompt, output_key="solutions")

/Users/dante/miniconda3/envs/langchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
evaluation_prompt = PromptTemplate(
    template="""
    제시된 해결 방법들을 각각 아래 항목으로 평가해줘
    * 해결 방법: {solutions}
    * 항목
        1) 장점
        2) 단점
        3) 투입 에너지 대비 영향도
        4) 현실성
    * 평가 결과: 
    """,
    input_variables=["solutions"]
)

evaluation_chain = LLMChain(llm=gpt35, prompt=evaluation_prompt, output_key="evaluations")

In [5]:
reasoning_prompt = PromptTemplate(
    template="""
    각 평가 결과를 보고 시나리오, 구현 전략, 필요한 준비사항, 잠재 위험 해결 방법에 대해 가장 효과적인 해결방법을 추론하여 설명해줘.
    * 평과 결과: {evaluations}
    * 향상된 추론 목록: 
    """,
    input_variables=["evaluations"]
)

reasoning_chain = LLMChain(llm=gpt35, prompt=reasoning_prompt, output_key="improved_reasonings")

In [6]:
ranking_prompt = PromptTemplate(
    template="""
    평가결과와 추론결과에 기반하여, 각 해결방법들의 순위를 매겨 순서대로 나열해줘.
    * 향상된 추론 목록: {improved_reasonings}
    """,
    input_variables=["improved_reasonings"]
)

ranking_chain = LLMChain(llm=gpt35, prompt=ranking_prompt, output_key="rankings")

In [9]:
tot_chain = SequentialChain(
    chains = [solution_chain, evaluation_chain, reasoning_chain, ranking_chain],
    input_variables=["count", "problem", "factors"],
    output_variables=["rankings"]
)

resp = tot_chain.run(
    count=3,
    problem="데이터 직무로 입사를 준비하고 있어. 그런데 잘 준비를 못한것 같아. 어떻게 준비하면 좋을까?",
    factors="면접, 코딩테스트, 자기소개서, 포트폴리오"
)

pprint(resp)

/Users/dante/miniconda3/envs/langchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'1. 향상된 추론 목록\n2. 시나리오\n3. 구현 전략\n4. 필요한 준비사항\n5. 잠재 위험 해결 방법'


- 순서대로 나열되긴 했으나, 항목명만 있고, 내용이 없는  답변이 나왔다.
- Langsmith를 조회해보자.
- https://smith.langchain.com/

___
* Fine-Tuning
    - 방식 및 개념 : 특정 작업에 대해 모델을 세부적으로 재학습시키는 기법. 특정 데이터셋을 사용하여 모델의 성능을 향상시킴
    - 특징: 낮은 성능의 모델도 튜닝을 통해 해당 주제에 대한 높은 성능을 보장해줌. 단, 과적합을 방지하기 위한 적절한 규제가 필요

___
Fine-Tuning은 별도의 챕터에서 다루겠습니다.